##### Cargue de librerias

In [1]:
from pycaret.clustering import *

import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint

import warnings
warnings.filterwarnings("ignore")

##### Conexion al ERP/DWH de Dispapeles y cargue de bds adicionales

In [ ]:
# Conexion al AS
cnxn = pyodbc.connect(
    driver='{iSeries Access ODBC Driver}',
    system='192.168.100.2',
    uid='TBLPRDUSR',
    pwd='bbbb')
cursor = cnxn.cursor()
#cnxn.close()

# Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute('SELECT DSTCOD AS "Codigo distrito", VTAZNA AS "Codigo zona", VTANIT AS "Nit cliente", ((2022 - LEFT(MAX(VTAFCH),4)) * 12 ) + (6 - SUBSTRING(MAX(VTAFCH),5,2)) AS "Recency", SUM(A.VTAVLRVTA) AS "Monetary", COUNT( DISTINCT LEFT(VTAFCH, 6)) AS "Frequency" FROM SIDPRDDTA.VTAHEC A WHERE PRMCOD = 1 AND A.DSTCOD = 10 AND A.VTAZNA < 96 AND LEFT(VTAFCH,6) BETWEEN 202011 AND 202210 GROUP BY DSTCOD, VTAZNA, VTANIT')
rows = cursor.fetchall()
df_rdo = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

SELECT 
	TMPDST AS "Codigo distrito",
	TMPLL4 AS "Codigo zona actual",
	CONCAT(CONCAT(TMPN5, '-'), TMPFLG3) AS "Nit cliente-sucursal",
	LEFT(TMPFEC, 4) AS "Ano",
	SUBSTRING(TMPFEC, 5, 2) AS "Mes",
	(((2021 - MAX(LEFT(TMPFEC, 4))) * 12 ) + (0 - MAX(SUBSTRING(TMPFEC, 5, 2)))) * -1 AS "Recency rcd",
	(SUM(TMPN1) + SUM(TMPN7) + SUM(TMPN10) + SUM(TMPN11) + SUM(TMPN14) + SUM(TMPN12)) AS "Monetary rcd",
	COUNT( DISTINCT SUBSTRING(TMPFEC, 5, 2)) AS "Frequency rcd"
FROM
	SIDPRDDTA.TMPRPT
WHERE	
	TMPRPT = 'PRCDVNDE' AND
	TMPPRM = 1 AND
	TMPDST NOT IN (30, 35) AND
	TMPLL4 < 96 
GROUP BY
	TMPDST,
	TMPLL4,
	CONCAT(CONCAT(TMPN5, '-'), TMPFLG3),
	LEFT(TMPFEC, 4),
	SUBSTRING(TMPFEC, 5, 2)
HAVING
	(SUM(TMPN1) + SUM(TMPN7) + SUM(TMPN10) + SUM(TMPN11) + SUM(TMPN14) + SUM(TMPN12)) > 0

In [2]:
## Conexion al DWH
cnxn = pyodbc.connect(
    driver='{SQL Server}',
    server='192.168.100.58',
    uid='bilectura',
    pwd='D1sp@p3l3s')
cursor = cnxn.cursor()
#cnxn.close()

In [3]:
codigo_SQL = "SELECT A.DSTCOD AS 'Codigo distrito', B.VNDZNA AS 'Codigo zona actual', CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC) AS 'Nit cliente-sucursal', A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', (((2021 - MAX(A.VTAANO)) * 12 ) + (0 - MAX(A.VTAMES))) * -1 AS 'Recency', SUM(A.VTAVLRVTA) AS 'Monetary', COUNT( DISTINCT A.VTAMES) AS 'Frequency' FROM V_VTA_VTAHEC A LEFT JOIN	DIM_CLIENTES B ON A.PRMCOD = B.PRMCOD AND A.DSTCOD = B.DSTCOD AND A.VTANIT = B.CLTNIT AND A.VTASUC = B.CLTSUC WHERE A.PRMCOD = 1 AND A.DSTCOD NOT IN (30,35) AND A.VTAZNA < 96 AND A.VTAFCH BETWEEN '2021-01-01' AND '2022-12-31' GROUP BY A.DSTCOD, B.VNDZNA, CONCAT(CONCAT(A.VTANIT,'-'),A.VTASUC),	A.VTAANO, A.VTAMES HAVING SUM(A.VTAVLRVTA) > 0"

In [4]:
#Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute(codigo_SQL)
rows = cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

In [5]:
df.head()

,Codigo distrito,Codigo zona actual,Nit cliente-sucursal,Ano,Mes,Recency,Monetary,Frequency
0,10,1,1077871537-1,2022,8,20,22599528.0000,1
1,10,1,17172243-1,2022,3,15,976752.0000,1
2,10,1,19322486-1,2021,2,2,1848088.2400,1
3,10,1,19322486-1,2022,1,13,4350769.7400,1
4,10,1,19322486-1,2022,8,20,1470204.9600,1


In [ ]:
maestra_zonas = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/Análisis clustering/Maestra_zonas.csv', delimiter= ";")

##### Tranformaciones

In [ ]:
df["Monetary"] = df["Monetary"].astype(int)

In [ ]:
df_2 = pd.merge(df, maestra_zonas, on= ["Codigo distrito", "Codigo zona actual"])

In [ ]:
list_filter = ['Artes graficas', 'V Horeca', 'Institucional', 'V SyE',
                'V Industria', 'Mayorista']

In [ ]:
df = df_2[df_2["Nombre tipo zona"].isin(list_filter)]

In [ ]:
df.describe()

In [ ]:
df_3 = df.groupby(["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal"]).agg({"Recency": np.max,
                                                                            "Monetary": np.sum,
                                                                            "Frequency": np.sum
                                                                            })
df_3 = df_3.reset_index(col_level= 1)
df_3

In [ ]:
#Se añade una key de Distrito-Nombretipozona para hacer mas facil la identificacion cuando se usa todo el pais
#Se genera la calificacion de Recency basado en los valores máximos
#Se divide por millón la columna Monetary para disminuir el rango de los valores
df_3['Distrito-Nombretipozona'] = df_3['Codigo distrito'].astype(str) + '-' + df_3['Nombre tipo zona']
df_3['Monetary'] = (df_3['Monetary'] / 1000).astype(int)
df_3['Frequency'] = (df_3['Frequency']).astype(int)

In [ ]:
df_3.describe()

##### Creacion, ajuste y descarga de los modelos de clustering por cada distrito-tipo de zona

In [ ]:
list_dtos = df_3["Distrito-Nombretipozona"].unique()

In [ ]:
list_dtos = ['10-Artes graficas', '10-Institucional']

In [ ]:
numeric_features = ["Recency", "Monetary", "Frequency"]
ignore_columns = ["Codigo distrito", "Nombre tipo zona", "Nit cliente-sucursal", "Distrito-Nombretipozona"]
normalizar = [False, True]

##### Dos modelos, norm= True y norm= False

In [ ]:
bd_predicted = pd.DataFrame([])
metrics = []
fecha = '1-2-23'

for norm in normalizar:
    for n, dto in enumerate(list_dtos):
        print(f"Procesando modelo normalizado {norm} {n+1} de {len(list_dtos)}. {(n+1) / (len(list_dtos)):.1%}")
        
        #BD filtrada
        df_dto = df_3[df_3["Distrito-Nombretipozona"] == dto]
        
        #Setup del modelo 
        s = setup(
                    df_dto,
                    normalize = norm,
                    verbose= False,
                    preprocess= False,
                    silent= True,
                    profile= False,
                    ignore_features= ignore_columns,
                    numeric_features= numeric_features
                    )
        
        #Creacion y prediccion de modelo
        model_kmeans = create_model(
                                    'kmeans',
                                    num_clusters= 5,
                                    verbose= False,
                                    )
        pull()
        df_pred_km = predict_model(
                                    model_kmeans,
                                    data = df_dto
                                    )
        
        # Reasignacion de los nombres de los clusters
        trans = df_pred_km.groupby(
                                    ["Cluster"]).agg({
                                                    "Recency": np.mean, "Monetary": np.mean, "Frequency": np.mean,
                                                    }).sort_values(
                                                                    by= ["Recency", "Frequency", "Monetary"],
                                                                    ascending= False
                                                                    ).reset_index()
        trans["Letra cluster"] = ["A", "B", "C", "D", "E"]
        trans = trans[["Cluster", "Letra cluster"]]
        
        df_pred_km = pd.merge(df_pred_km, trans, on= ["Cluster"])
        
        # Descargue del modelo
        bd_predicted = bd_predicted.append(df_pred_km)

        #Append a una base general
        metrics.append({
                        "Distrito-zona": dto,
                        "Normalizado": norm,
                        "Silhouette": pull().Silhouette[0],
                        "Calinski-Harabasz": pull()["Calinski-Harabasz"][0],
                        "Davies-Bouldin": pull()["Davies-Bouldin"][0]
                        })
        # save_model(model_kmeans, f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Modelos {fecha}/Modelo {dto} {fecha}")

metrics = pd.DataFrame(metrics)
metrics["Normalizado"] = metrics["Normalizado"].replace([True, False], ["Si","No"])
metrics["Tipo zona"] = metrics["Distrito-zona"].str[3:]

print("Proceso completo 100%")

##### Un solo modelo, norm= False

In [ ]:
bd_predicted = pd.DataFrame([])
metrics = []
fecha = '1-2-23'

for n, dto in enumerate(list_dtos):
    print(f"Procesando modelo {n+1} de {len(list_dtos)}. {(n+1) / (len(list_dtos)):.1%}")
    
    #BD filtrada
    df_dto = df_3[df_3["Distrito-Nombretipozona"] == dto]
    
    #Setup del modelo 
    s = setup(
                df_dto,
                normalize = False,
                verbose= False,
                preprocess= False,
                silent= True,
                profile= False,
                ignore_features= ignore_columns,
                numeric_features= numeric_features
                )
    
    #Creacion y prediccion de modelo
    model_kmeans = create_model(
                                'kmeans',
                                num_clusters= 5,
                                verbose= False,
                                )
    pull()
    df_pred_km = predict_model(
                                model_kmeans,
                                data = df_dto
                                )
    
    # Reasignacion de los nombres de los clusters
    trans = df_pred_km.groupby(
                                ["Cluster"]).agg({
                                                "Recency": np.mean, "Monetary": np.mean, "Frequency": np.mean,
                                                }).sort_values(
                                                                by= ["Recency", "Frequency", "Monetary"],
                                                                ascending= False
                                                                ).reset_index()
    trans["Letra cluster"] = ["A", "B", "C", "D", "E"]
    trans = trans[["Cluster", "Letra cluster"]]
    
    df_pred_km = pd.merge(df_pred_km, trans, on= ["Cluster"])
    
    # Descargue del modelo
    bd_predicted = bd_predicted.append(df_pred_km)

    #Append a una base general
    metrics.append({
                    "Distrito-zona": dto,
                    "Silhouette": pull().Silhouette[0],
                    "Calinski-Harabasz": pull()["Calinski-Harabasz"][0],
                    "Davies-Bouldin": pull()["Davies-Bouldin"][0]
                    })
    # save_model(model_kmeans, f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Modelos {fecha}/Modelo {dto} {fecha}")

metrics = pd.DataFrame(metrics)
metrics["Tipo zona"] = metrics["Distrito-zona"].str[3:]

print("Proceso completo 100%")

In [ ]:
metrics

In [ ]:
f, axarr = plt.subplots(2,3)
f.set_figheight(10)
f.set_figwidth(25)

metricas = ["Silhouette", "Calinski-Harabasz", "Davies-Bouldin"]

for n, met in enumerate(metricas):
    f.add_subplot(axarr[0,n])
    plt.title(f"Distribución {met}").set_fontsize(15)
    sns.set_theme(style="darkgrid", palette= "pastel")
    sns.boxplot(data= metrics, x= met, y= "Tipo zona", hue= 'Normalizado')

    f.add_subplot(axarr[1,n])
    sns.set_theme(style="darkgrid", palette= "pastel")
    sns.histplot(data= metrics, x= met, hue= 'Normalizado')
plt.show()



In [ ]:
metrics[metrics['Normalizado'] == "Si"].describe()

In [ ]:
metrics[metrics['Normalizado'] == "No"].describe()

In [ ]:
#Descargue de bd predicha y metricas calculadas
bd_predicted.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",", sep= ";")
metrics.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Metricas {fecha}.csv",
                        encoding= 'utf-8', index= False, decimal= ",",  sep= ";")

In [ ]:
fig = plt.figure(figsize=(7,10))
ax = fig.gca(projection='3d')

m = ['o', 'v', 's', 'p', '*']
col =  ['blueviolet', 'limegreen', 'indianred', 'royalblue', 'magenta']
leg = ['Cluster 0','Cluster 1','Cluster 2','Cluster 3', 'Cluster 4']
cluster = list(np.unique(df_pred_km['Cluster']))

for c in range(5):
    df_plot = df_pred_km[df_pred_km['Cluster'] == cluster[c]]
    ax.scatter(df_plot['Monetary'],df_plot['Frequency'], df_plot['Recency'] , marker= m[c], c= col[c])
    
ax.legend(leg)
ax.set_xlabel('Monetary')
ax.set_ylabel('Recency')
ax.set_zlabel('Frequency')

In [ ]:
plot_model(model_kmeans, plot = 'cluster')

In [ ]:
# saved_model =  load_model('Model Version random 9-11')